<a href="https://colab.research.google.com/github/Mc4minta/AIB5-PcapAttackClassifier/blob/main/EvaluateModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# CICFlowMeterSetup

In [ ]:
!sudo apt-get install libpcap-dev # install libpcap-dev
!wget https://codeberg.org/iortega/TCPDUMP_and_CICFlowMeter/archive/master:CICFlowMeters/CICFlowMeter-3.0.zip # download CICFlowMeter3.0.zip
!unzip CICFlowMeter-3.0.zip -d CICFlowMeter-3.0 # Extract CICFlowMeter3.0.zip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libdbus-1-dev libpcap0.8 libpcap0.8-dev
The following NEW packages will be installed:
  libdbus-1-dev libpcap-dev libpcap0.8 libpcap0.8-dev
0 upgraded, 4 newly installed, 0 to remove and 35 not upgraded.
Need to get 607 kB of archives.
After this operation, 2,238 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpcap0.8 amd64 1.10.1-4ubuntu1.22.04.1 [145 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libdbus-1-dev amd64 1.12.20-2ubuntu4.1 [188 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpcap0.8-dev amd64 1.10.1-4ubuntu1.22.04.1 [270 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpcap-dev amd64 1.10.1-4ubuntu1.22.04.1 [3,326 B]
Fetched 607 kB in 1s (485 kB/s)
debconf: unable to initialize frontend: Dialog
deb

In [ ]:
import os

# Define the directory paths
in_directory = 'data/in'
out_directory = 'data/out'

# Create the directories if they don't exist
os.makedirs(in_directory, exist_ok=True)
os.makedirs(out_directory, exist_ok=True)

print(f"Directories '{in_directory}' and '{out_directory}' created (if they didn't exist).")

Directories 'data/in' and 'data/out' created (if they didn't exist).


# Setup attack_csv

In [135]:
import os
import zipfile

def list_csv_files(attack):

  # list attack_csv.zip file
  attack = attack + '_csv'
  directory_path = '/content/drive/MyDrive/Share to Mc4/AIBuilders5-MiN/TestsetForbaseline_csv'

  if os.path.exists(directory_path):
    contents = os.listdir(directory_path)
    for item in contents:
      print(item)
  else:
    print(f"Directory not found: {directory_path}")


  print("\n-------------------------\n")
  # unzip attack_csv.zip file
  zip_file_path = '/content/drive/MyDrive/Share to Mc4/AIBuilders5-MiN/TestsetForbaseline_csv/' + attack + '.zip'
  destination_directory = '/content/'

  try:
      with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
          zip_ref.extractall(destination_directory)
      print(f"'{zip_file_path}' unzipped to '{destination_directory}'.")
  except FileNotFoundError:
      print(f"Error: Zip file not found at '{zip_file_path}'")
  except zipfile.BadZipFile:
      print(f"Error: '{zip_file_path}' is not a valid zip file.")
  except Exception as e:
      print(f"An error occurred: {e}")

  print("\n-------------------------\n")
  # list unzipped attack_csv files
  directory = attack
  csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]

  if csv_files:
      print(f"Found {len(csv_files)} .csv files in '{directory}':")
      for csv_file in csv_files:
          print(csv_file)
  else:
      print(f"No .csv files found in '{directory}'.")

# list_csv_files(attack)

# Load model

In [137]:
import joblib
def load_model(model_name):
  model_path = '/content/drive/MyDrive/Share to Mc4/AIBuilders5-MiN/' + model_name
  model = joblib.load(model_path)
  return model

# model = load_model(model_name)
# print(model)

# Concat and Label same type csv and load as dataframe

In [138]:
import pandas as pd
import os

def concat_csv(attack):
  # list unzipped attack_csv files
  attack_folder = attack + '_csv'
  directory = attack_folder
  csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]

  all_csv_data = []

  # add attack to label and concat csvs
  if csv_files:
      print(f"Found {len(csv_files)} .csv files in '{directory}':")
      for csv_file in csv_files:
          print(csv_file)
          file_path = os.path.join(directory, csv_file)
          try:
              df = pd.read_csv(file_path)
              df['Label'] = attack
              all_csv_data.append(df)
          except Exception as e:
              print(f"Error reading {csv_file}: {e}")

      print("\n-------------------------\n")
      if all_csv_data:
          concatenated_df = pd.concat(all_csv_data, ignore_index=True)
          print("All CSV files concatenated into a single DataFrame.")
          return concatenated_df
      else:
          print("No data read from CSV files.")
          return pd.DataFrame()
  else:
      print(f"No .csv files found in '{directory}'.")
      return pd.DataFrame()

# Pre process dataframe

In [139]:
import pandas as pd
import numpy as np

def map_port(port):
    if port == 21:
        return 1  # FTP
    elif port == 22:
        return 2  # SSH
    elif port == 53:
        return 3  # DNS
    elif port == 80:
        return 4  # HTTP
    elif port == 443:
        return 5  # HTTPS
    else:
        return 6  # Other

def preprocess_dataframe(df):
    # replace space in columns name with underscore
    df.columns = df.columns.str.strip().str.replace(' ', '_')

    # drop objects type columns
    columns_to_drop = [
        'Flow_ID','Src_IP','Dst_IP','Src_Port','Protocol','Timestamp'
    ]

    df = df.drop(columns=columns_to_drop)

    # remove rows with missing and infinite values
    df.replace([np.inf, -np.inf], np.nan, inplace = True)
    df = df.dropna()

    # map destination port to 1-6 numbers
    df['Dst_Port'] = df['Dst_Port'].apply(map_port)

    return df

# preprocess_dataframe(df)

# Make Prediction

In [151]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd # Import pandas to display confusion matrix nicely
from IPython.display import display # Import display

def make_prediction(model, df):
  # Split the dataframe into features (X) and labels (y)
  X = df.drop('Label', axis=1)
  y = df['Label']

  # Make predictions
  predictions = model.predict(X)

  # Calculate and print the classification report
  print(classification_report(y, predictions))

  # Calculate and print the confusion matrix
  cm = confusion_matrix(y, predictions, labels=model.classes_)
  cm_df = pd.DataFrame(cm, index=model.classes_, columns=model.classes_)
  print("\nConfusion Matrix:")
  display(cm_df) # Use display() instead of print()

# make_prediction(model, df)

# Main

In [152]:
attack = 'SSH-Bruteforce'
model_name = 'RandomForest400IntPortCIC1718-2.pkl'

model = load_model(model_name)
attacks = model.classes_

for attack in attacks:
  list_csv_files(attack)
  df = concat_csv(attack)
  df = preprocess_dataframe(df)
  print(f"Prediction of {attack}")
  make_prediction(model, df)

DoS_csv.zip
DDoS_csv.zip
PortScan_csv.zip
Benign_csv.zip
DoS-HTTP-Flood_csv.zip
DoS-Layer3and4_csv.zip
DoS-overall_csv.zip
FTP-Bruteforce_csv.zip
SSH-Bruteforce_csv.zip
DoS-Slow-Rate_csv.zip

-------------------------

'/content/drive/MyDrive/Share to Mc4/AIBuilders5-MiN/TestsetForbaseline_csv/Benign_csv.zip' unzipped to '/content/'.

-------------------------

Found 21 .csv files in 'Benign_csv':
normal_10_ISCX.csv
Normal-h3_11_ISCX.csv
DoS-Hulk_benign_ISCX.csv
Normal-h3_6_ISCX.csv
normal_12_ISCX.csv
Normal-h3_2_ISCX.csv
Normal-h3_10_ISCX.csv
normal_3_ISCX.csv
normal_2_ISCX.csv
Normal-h3_1_ISCX.csv
Normal-h3_5_ISCX.csv
normal_11_ISCX.csv
Normal-h3_4_ISCX.csv
Normal-h3_3_ISCX.csv
Normal-h3_8_ISCX.csv
normal_4_ISCX.csv
normal_13_ISCX.csv
Normal-h3_12_ISCX.csv
Normal-h3_9_ISCX.csv
DoS-GoldenEye_benign_ISCX.csv
normal_1_ISCX.csv
Found 21 .csv files in 'Benign_csv':
normal_10_ISCX.csv
Normal-h3_11_ISCX.csv
DoS-Hulk_benign_ISCX.csv
Normal-h3_6_ISCX.csv
normal_12_ISCX.csv
Normal-h3_2_ISCX.cs

,Benign,DoS-HTTP-Flood,DoS-Slow-Rate,FTP-Bruteforce,PortScan,SSH-Bruteforce
Benign,58684,0,1,0,0,1
DoS-HTTP-Flood,0,0,0,0,0,0
DoS-Slow-Rate,0,0,0,0,0,0
FTP-Bruteforce,0,0,0,0,0,0
PortScan,0,0,0,0,0,0
SSH-Bruteforce,0,0,0,0,0,0


DoS_csv.zip
DDoS_csv.zip
PortScan_csv.zip
Benign_csv.zip
DoS-HTTP-Flood_csv.zip
DoS-Layer3and4_csv.zip
DoS-overall_csv.zip
FTP-Bruteforce_csv.zip
SSH-Bruteforce_csv.zip
DoS-Slow-Rate_csv.zip

-------------------------

'/content/drive/MyDrive/Share to Mc4/AIBuilders5-MiN/TestsetForbaseline_csv/DoS-HTTP-Flood_csv.zip' unzipped to '/content/'.

-------------------------

Found 6 .csv files in 'DoS-HTTP-Flood_csv':
DoS-Hulk_attack_ISCX.csv
DoS-GoldenEye_attack_ISCX.csv
GoldenEye_ISCX.csv
hulk_ISCX.csv
DDoS HTTP Flood Attacks_ISCX.csv
http_dos_ISCX.csv
Found 6 .csv files in 'DoS-HTTP-Flood_csv':
DoS-Hulk_attack_ISCX.csv
DoS-GoldenEye_attack_ISCX.csv
GoldenEye_ISCX.csv
hulk_ISCX.csv
DDoS HTTP Flood Attacks_ISCX.csv
http_dos_ISCX.csv

-------------------------

All CSV files concatenated into a single DataFrame.
Prediction of DoS-HTTP-Flood
                precision    recall  f1-score   support

        Benign       0.00      0.00      0.00         0
DoS-HTTP-Flood       1.00      0.04     

,Benign,DoS-HTTP-Flood,DoS-Slow-Rate,FTP-Bruteforce,PortScan,SSH-Bruteforce
Benign,0,0,0,0,0,0
DoS-HTTP-Flood,179010,7019,8642,0,0,0
DoS-Slow-Rate,0,0,0,0,0,0
FTP-Bruteforce,0,0,0,0,0,0
PortScan,0,0,0,0,0,0
SSH-Bruteforce,0,0,0,0,0,0


DoS_csv.zip
DDoS_csv.zip
PortScan_csv.zip
Benign_csv.zip
DoS-HTTP-Flood_csv.zip
DoS-Layer3and4_csv.zip
DoS-overall_csv.zip
FTP-Bruteforce_csv.zip
SSH-Bruteforce_csv.zip
DoS-Slow-Rate_csv.zip

-------------------------

'/content/drive/MyDrive/Share to Mc4/AIBuilders5-MiN/TestsetForbaseline_csv/DoS-Slow-Rate_csv.zip' unzipped to '/content/'.

-------------------------

Found 4 .csv files in 'DoS-Slow-Rate_csv':
slowread_1_ISCX.csv
slowread_2_ISCX.csv
lowrateddosMQTT_ISCX.csv
http_slowloris_ISCX.csv
Found 4 .csv files in 'DoS-Slow-Rate_csv':
slowread_1_ISCX.csv
slowread_2_ISCX.csv
lowrateddosMQTT_ISCX.csv
http_slowloris_ISCX.csv

-------------------------

All CSV files concatenated into a single DataFrame.
Prediction of DoS-Slow-Rate
               precision    recall  f1-score   support

       Benign       0.00      0.00      0.00         0
DoS-Slow-Rate       1.00      0.02      0.04     51652

     accuracy                           0.02     51652
    macro avg       0.50      0.01 

...

,Benign,DoS-HTTP-Flood,DoS-Slow-Rate,FTP-Bruteforce,PortScan,SSH-Bruteforce
Benign,0,0,0,0,0,0
DoS-HTTP-Flood,0,0,0,0,0,0
DoS-Slow-Rate,0,0,0,0,0,0
FTP-Bruteforce,8497,0,0,903,0,0
PortScan,0,0,0,0,0,0
SSH-Bruteforce,0,0,0,0,0,0


DoS_csv.zip
DDoS_csv.zip
PortScan_csv.zip
Benign_csv.zip
DoS-HTTP-Flood_csv.zip
DoS-Layer3and4_csv.zip
DoS-overall_csv.zip
FTP-Bruteforce_csv.zip
SSH-Bruteforce_csv.zip
DoS-Slow-Rate_csv.zip

-------------------------

'/content/drive/MyDrive/Share to Mc4/AIBuilders5-MiN/TestsetForbaseline_csv/PortScan_csv.zip' unzipped to '/content/'.

-------------------------

Found 10 .csv files in 'PortScan_csv':
nmap_fin_ISCX.csv
normal_scanning4_ISCX.csv
normal_scanning6_ISCX.csv
normal_scanning3_ISCX.csv
nmap_null_ISCX.csv
normal_scanning1_ISCX.csv
portscan_ISCX.csv
Port Scanning attack_ISCX.csv
normal_scanning5_ISCX.csv
normal_scanning2_ISCX.csv
Found 10 .csv files in 'PortScan_csv':
nmap_fin_ISCX.csv
normal_scanning4_ISCX.csv
normal_scanning6_ISCX.csv
normal_scanning3_ISCX.csv
nmap_null_ISCX.csv
normal_scanning1_ISCX.csv
portscan_ISCX.csv
Port Scanning attack_ISCX.csv
normal_scanning5_ISCX.csv
normal_scanning2_ISCX.csv

-------------------------

All CSV files concatenated into a single DataF

,Benign,DoS-HTTP-Flood,DoS-Slow-Rate,FTP-Bruteforce,PortScan,SSH-Bruteforce
Benign,0,0,0,0,0,0
DoS-HTTP-Flood,0,0,0,0,0,0
DoS-Slow-Rate,0,0,0,0,0,0
FTP-Bruteforce,0,0,0,0,0,0
PortScan,146127,0,0,0,0,0
SSH-Bruteforce,0,0,0,0,0,0


DoS_csv.zip
DDoS_csv.zip
PortScan_csv.zip
Benign_csv.zip
DoS-HTTP-Flood_csv.zip
DoS-Layer3and4_csv.zip
DoS-overall_csv.zip
FTP-Bruteforce_csv.zip
SSH-Bruteforce_csv.zip
DoS-Slow-Rate_csv.zip

-------------------------

'/content/drive/MyDrive/Share to Mc4/AIBuilders5-MiN/TestsetForbaseline_csv/SSH-Bruteforce_csv.zip' unzipped to '/content/'.

-------------------------

Found 3 .csv files in 'SSH-Bruteforce_csv':
bruteforce_ssh_ISCX.csv
sshbrute-kali_ISCX.csv
sshbrute-ubuntu_ISCX.csv
Found 3 .csv files in 'SSH-Bruteforce_csv':
bruteforce_ssh_ISCX.csv
sshbrute-kali_ISCX.csv
sshbrute-ubuntu_ISCX.csv

-------------------------

All CSV files concatenated into a single DataFrame.
Prediction of SSH-Bruteforce
                precision    recall  f1-score   support

        Benign       0.00      0.00      0.00         0
DoS-HTTP-Flood       0.00      0.00      0.00         0
      PortScan       0.00      0.00      0.00         0
SSH-Bruteforce       1.00      0.42      0.60      7225

     

,Benign,DoS-HTTP-Flood,DoS-Slow-Rate,FTP-Bruteforce,PortScan,SSH-Bruteforce
Benign,0,0,0,0,0,0
DoS-HTTP-Flood,0,0,0,0,0,0
DoS-Slow-Rate,0,0,0,0,0,0
FTP-Bruteforce,0,0,0,0,0,0
PortScan,0,0,0,0,0,0
SSH-Bruteforce,4137,2,0,0,16,3070
